<a href="https://colab.research.google.com/github/sanchobarriga/covidpr.info/blob/master/AutomatedAlignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/My Drive')

In [ ]:
#os.mkdir('COVIDPRData')
os.chdir('/content/drive/My Drive/COVIDPRData')

In [ ]:
!pip install PyPDF2

     |████████████████████████████████| 81kB 2.2MB/s 
  Created wheel for PyPDF2: filename=PyPDF2-1.26.0-cp36-none-any.whl size=61086 sha256=465067f0f1ec99a6a8fe5341a10c0e9688f978a74fa9a3581d3954115b4cc516
  Stored in directory: /root/.cache/pip/wheels/53/84/19/35bc977c8bf5f0c23a8a011aa958acd4da4bbd7a229315c1b7
Successfully built PyPDF2


In [ ]:
from PyPDF2 import PdfFileReader
import pandas as pd

In [ ]:
from datetime import datetime
meses = ['enero', 'febrero','marzo','abril','mayo','junio','julio','agosto','septiembre','octubre','noviembre','diciembre']
a = datetime.now()
date = a.strftime("%Y%m%d")
dia = str(a.day)
mes = meses[a.month-1]
annum = str(a.year)
try:
  os.mkdir(date)
except:
  1
os.chdir('/content/drive/My Drive/COVIDPRData')
os.chdir(date)
url = 'http://www.salud.gov.pr/Estadisticas-Registros-y-Publicaciones/Estadsticas%20Casos%20Positivos%20COVID19%20'+mes+'%20'+annum+'/Informe%20Casos%20Positivos%20COVID-19%20('+dia+'%20'+mes+'%20'+annum+').pdf'
print('Downloading:',end=' ')
print(url)

import requests

r = requests.get(url, allow_redirects=True)

open('report.pdf', 'wb').write(r.content)

f = open('report.pdf', 'rb')
pdf = PdfFileReader(f)
information = pdf.getDocumentInfo()
number_of_pages = pdf.getNumPages()
print('Report length:',end=' ')
print(number_of_pages)
if number_of_pages == 12:
  vals = [4,7,8,9,10,11]
elif number_of_pages == 11:
  vals = [3,6,7,8,9,10]
names = ['PCR', 'Sero', 'IgM', 'IgM-IgG', 'IgG', 'Deaths']
lims = [1200,600,600,600,600,50]
outs = []
for k in range(len(vals)):
    i = vals[k]
    rt = pdf.getPage(i)
    text = rt.extractText()
    listed = text.split('\n')
    nums = []
    for j in listed:
        try:
            nums += [int(j)]
            if nums[-1] - nums[-2] > lims[k]:
                break
        except:
            1
    outs += [nums]

for j in range(len(names)):
    #os.chdir(r'C:\Users\Masters-PC\Documents\GraduateSchool\COVID19\JAMA\ReportesSalud')
    i = names[j]
    o = pd.DataFrame(outs[j])
#    try:
#        os.mkdir(pdf_path[:-4])
#    except:
#        1
#    os.chdir(pdf_path[:-4])
    o.to_csv (i+'.csv', index = None)


Downloading: http://www.salud.gov.pr/Estadisticas-Registros-y-Publicaciones/Estadsticas%20Casos%20Positivos%20COVID19%20julio%202020/Informe%20Casos%20Positivos%20COVID-19%20(5%20julio%202020).pdf
Report length: 12


In [ ]:
vals = [2,5]
names = ['PCRs','Seros']
lims = [600,600,600,600,600,50]
outs = []
for k in range(len(vals)):
    if i in vals:
        rt = pdf.getPage(i)
        text = rt.extractText()
        listed = text.split('\n')
        stars = []
        mnums = []
        for bt in range(len(listed)):
            qt = listed[bt]
            if '*' in qt:
                stars += [qt]
                mnums += [listed[bt+2]]
        table = pd.DataFrame(columns = ['Town', 'Value'])
        table.Town = stars
        table.Value = mnums
        table.to_csv('Muni' + names[k] +'.csv')


In [ ]:
print(names)
print(outs[0])

['PCR', 'Sero', 'IgM', 'IgM-IgG', 'IgG', 'Deaths']
[1, 3, 3, 9, 7, 5, 5, 14, 11, 10, 34, 48, 41, 57, 49, 24, 12, 84, 56, 76, 53, 50, 15, 3, 51, 33, 23, 34, 3, 12, 4, 28, 24, 30, 27, 21, 9, 2, 18, 19, 27, 17, 16, 1, 3, 14, 15, 19, 23, 9, 6, 3, 6, 22, 20, 18, 23, 4, 15, 10, 9, 9, 11, 2, 2, 7, 5, 12, 5, 6, 3, 2, 5, 5, 8, 20, 27, 2, 1, 25, 17, 7, 9, 9, 3, 2, 9, 12, 14, 11, 15, 11, 7, 17, 18, 9, 18, 12, 3, 24, 33, 16, 26, 28, 5, 2, 46, 17, 1, 1846]


In [ ]:
import numpy as np
def nnz(cand):
    #Find next non-zero number in a vector
    return list(np.array(cand)-1==-1).index(False)
def interPad(a2, N):
    outs = [a2]
    for i in range(N):
        nouts = []
        for j in outs:
            for k in range(len(j)):
                nouts += [np.insert(j, k, 0)]
        outs = nouts
    for j in range(len(outs)):
        outs[j] = list(outs[j])
    return outs

def lazyPad(a1, a2):
    zseq = np.append(a1,0)
    start = list((zseq-1)==-1).index(False)
    zseq = zseq[start:]
    target = len(a1)+1
    zseqs = interPad(a2,target-len(a2))
    sqdiff = []
    for k in range(len(zseqs)):
        i = zseqs[k]
        sqdiff += [sum((i-zseq)**2)]
    return zseqs[sqdiff.index(min(sqdiff))]
    
def spreadPad(a1, a2):
    count = 0 
    zseq = np.append(a1,0)
    bseq = np.append(a1,0)
    start = -1
    outtie = [0]*len(zseq)
    for j in range(len(a2)):
        i = a2[j]
        if sum(bseq) == 0:
            for e in range(len(a2)-j):
                outtie[start+e+1] = a2[j+e]
            break
        k = list((bseq-1)==-1).index(False)
        val = bseq[k]
        if i == val:
            count += 1
            bseq = bseq[k+1:]
            start = list(zseq).index(val, start + 1)
            outtie[start] = i

        else:
            if ((count > 2)  & (k > 0)):
                cands = interPad(a2[j:j+5-k],k)
                if j + 5 - k > len(a2):
                    lim = (j+5-k)-len(a2)
                else:
                    lim = 0
                vals = []
                for cn in range(len(cands)):
                    cand = cands[cn]
                    vals += [sum((zseq[start+1:start+6-lim]-np.array(cand))**2)]
                    #print(vals)
                win = vals.index(min(vals))
                cand = cands[win]
                if nnz(cand) == 1:
                    bseq = bseq[k+1:]
                    start = list(zseq).index(val, start + 1)
                    outtie[start] = i
                else:
                    print('requires attention-1')
                    break
            elif (abs(i - val)/val) < 0.5:
                count += 1
                bseq = bseq[k+1:]
                start = list(zseq).index(val, start + 1)
                outtie[start] = i
                continue
            else:
                for e in range(len(a2)-j):
                    outtie[start+e+1] = a2[j+e]
                print('requires attention-2')
                break
    return outtie


In [ ]:
print(list(old))
print(list(new))
print('')
print('')

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 4, 0, 1, 1, 2, 1, 4, 6, 3, 0, 0, 1, 0, 12, 8, 10, 11, 9, 4, 0, 13, 20, 14, 21, 13, 4, 1, 9, 10, 18, 20, 29, 10, 3, 14, 20, 20, 19, 16, 7, 0, 18, 23, 9, 15, 15, 9, 1, 14, 19, 14, 11, 28, 6, 3, 8, 15, 14, 17, 34, 2, 4, 21, 18, 24, 9, 21, 14, 1, 11, 12, 7, 5, 5, 6, 1, 8, 7, 16, 15, 10, 4, 0, 8, 14, 11, 15, 9, 2, 0, 13, 8, 3, 0, 0, 0]
[4, 1, 1, 2, 1, 4, 6, 3, 1, 12, 8, 10, 11, 9, 4, 13, 20, 14, 21, 13, 4, 1, 9, 10, 18, 20, 29, 10, 3, 14, 20, 20, 19, 16, 7, 18, 23, 9, 16, 15, 9, 1, 14, 19, 14, 11, 28, 6, 3, 8, 15, 14, 17, 34, 2, 4, 21, 18, 24, 9, 28, 14, 1, 11, 12, 7, 6, 5, 6, 1, 8, 7, 17, 15, 10, 4, 8, 14, 12, 15, 11, 2, 14, 13, 8]




In [ ]:
os.chdir('/content/drive/My Drive/COVIDPRData')
yest = '20200704'
today = '20200705'
L = os.listdir(today)
curves = pd.read_csv(yest+'/Curves.csv', encoding = 'utf-16', sep=None)
print(L)
print(os.getcwd())
newcurve = pd.DataFrame()
for k in range(len(L)):
  i = L[k]
  if '.csv' not in i:
    continue
  if i in ['MuniPCRs.csv', 'MuniSeros.csv','Deaths.csv','report.pdf', 'Curves.csv']:
    continue
  var = i[0:i.index('.')]
  new = pd.read_csv(today+'/'+i)
  new = new['0'][:-1]
  old = curves[var]
  if var == 'Sero':
    if new[0]==11:
      new = new[1:]
  optpad = spreadPad(np.array(old),np.array(new))
  print(optpad)
  newcurve[var] = optpad
export_csv = newcurve.to_csv (today+'/Curves.csv', index = None, header=True, encoding = 'utf-16') #Don't forget to add '.csv' at the end of the path

['report.pdf', 'PCR.csv', 'Sero.csv', 'IgM.csv', 'IgM-IgG.csv', 'IgG.csv', 'Deaths.csv']
/content/drive/My Drive/COVIDPRData
requires attention-2
[0, 1, 0, 0, 0, 3, 3, 0, 9, 7, 5, 5, 14, 11, 10, 34, 48, 41, 57, 49, 24, 12, 84, 56, 76, 53, 50, 15, 3, 51, 33, 23, 34, 3, 12, 4, 28, 24, 30, 27, 21, 9, 2, 18, 19, 27, 17, 16, 1, 3, 14, 15, 19, 23, 9, 6, 3, 6, 22, 20, 18, 23, 4, 15, 0, 10, 9, 9, 11, 2, 2, 7, 5, 12, 5, 6, 3, 2, 5, 5, 8, 20, 27, 2, 1, 25, 17, 7, 9, 9, 3, 2, 9, 12, 14, 11, 15, 11, 7, 17, 18, 9, 18, 12, 3, 0, 24, 33, 16, 26, 28, 5, 2, 46, 17, 1, 0, 0, 0, 0]
requires attention-2
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 18, 28, 0, 0, 3, 10, 8, 9, 11, 3, 1, 15, 26, 43, 18, 7, 6, 3, 30, 54, 49, 48, 48, 17, 3, 72, 75, 88, 80, 95, 26, 7, 85, 86, 81, 100, 125, 30, 16, 124, 145, 116, 140, 151, 32, 8, 115, 134, 106, 137, 169, 49, 9, 105, 105, 101, 84, 168, 57, 25, 56, 127, 113, 114, 131, 31, 15, 148, 101, 125, 116, 124, 43, 14, 88, 94, 70, 71, 84, 33, 8, 80, 60, 97, 74, 63, 